<a href="https://colab.research.google.com/github/yjyjy131/AI_Competitions_2/blob/main/simulation/boj23291_%EC%96%B4%ED%95%AD%EC%A0%95%EB%A6%AC_p5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import deque


# 잘라낸 부위를 회전
def transpose(arr):
    h = len(arr)
    w = len(arr[0])
    trans = [deque() for _ in range(w)]
    for i in range(h):
        for j in range(w):
            trans[j].appendleft(arr[i][j])
    return trans


# 옮길 부위를 자른 후 붙임
def split_bowl(arr):
    w = len(arr[0])
    max_w = len(arr[-1])

    # 자른다
    target_q = deque([arr[-1][i]for i in range(0, w)])
    remain_q = deque(arr[-1][i]for i in range(w, max_w))

    tmp_arr = arr[0:-1]
    tmp_arr.append(target_q)
    trans_target = arr[0:-1] + [target_q]

    # 회전
    new_target = transpose(trans_target)

    # 붙인다
    return new_target + [remain_q]


# 공중 부양 1
def move1(bowl_q):
    t_arr = [deque([bowl_q.popleft()]), bowl_q]

    while 1:
        # 가장 왼쪽 어항 자르고 붙임
        t_arr = split_bowl(t_arr)

        # 계산 가능한 어항 바닥이 남을 때까지 과정 반복
        h = len(t_arr)
        w = len(t_arr[0])
        if h > len(t_arr[-1]) - w:
            break

    return t_arr


# 물고기 수 조절
def move_fish(bowl_q):

    # 현재 위치 에서 탐색 가능한 좌표쌍 모두 찾음
    # 각 후보 좌표 sort 후 set 에 넣어 중복 제거
    candidates = []
    diff = []
    gox = [1, -1, 0, 0]
    goy = [0, 0, -1, 1]
    for x in range(len(bowl_q)):
        for y in range(len(bowl_q[x])):
            for dx, dy in zip(gox, goy):
                nx, ny = x + dx, y + dy
                if 0 <= nx < len(bowl_q):
                    if 0 <= ny < len(bowl_q[nx]):
                        tmp = sorted([(x, y), (nx, ny)])
                        if tmp not in candidates:
                            candidates.append(tmp)
                            # 차이 값 계산
                            calc = abs(bowl_q[x][y] - bowl_q[nx][ny]) // 5
                            if calc > 0:
                                if bowl_q[x][y] > bowl_q[nx][ny]:
                                    diff.append([(x, y), (nx, ny), calc])
                                else:
                                    diff.append([(nx, ny), (x, y), calc])

    # 물고기 이동
    for coord1, coord2, val in diff:
        bowl_q[coord1[0]][coord1[1]] -= val
        bowl_q[coord2[0]][coord2[1]] += val

    return bowl_q


# 일렬로 만듦
def stretch_bowl(bowl_q):
    w = len(bowl_q[0])
    max_w = len(bowl_q[-1])

    # 자른다
    target_q = deque([bowl_q[-1][i] for i in range(0, w)])
    remain_q = deque(bowl_q[-1][i] for i in range(w, max_w))

    tmp_arr = bowl_q[0:-1]
    tmp_arr.append(target_q)
    trans_target = bowl_q[0:-1] + [target_q]

    trans_q = transpose(trans_target)

    res_list = []
    for q in trans_q:
        for _ in range(len(q)):
            res_list.append(q.popleft())

    for _ in range(len(remain_q)):
        res_list.append(remain_q.popleft())

    return deque(res_list)


# 공중 부양 2
def move2(bowl_q):
    # 가운데 자르고 시계 방향 돌림
    w = len(bowl_q)

    # 가운데 자르고 180도 회전
    split1 = deque([])
    for j in range(w//2):
        split1.appendleft(bowl_q.popleft())

    # 위에 붙임
    t_bowl = [split1, bowl_q]

    # 가운데 자르고 회전시키고 
    h = len(t_bowl)
    w = len(t_bowl[0])
    split2 = deque([])

    for i in range(h):
        tmp = deque()
        for j in range(w//2):
            tmp.appendleft(t_bowl[i].popleft())
        split2.appendleft(tmp)

    # 위에 붙임
    res_bowl = []
    for s in split2:
        res_bowl.append(s)
    res_bowl += t_bowl

    return res_bowl


if __name__ == '__main__':
    n, k = map(int, input().split())
    bowl = deque(map(int, input().split()))
    ans = 0

    while 1:

        # 물꼬기 수 차이를 구함
        if max(bowl) - min(bowl) <= k:
            break
        ans += 1
        # 물고기 수가 가장 적은 어항에 물고기 한 마리 넣음
        # 여러 개면 모두 한마리 씩 넣음
        min_v = min(bowl)
        for i, v in enumerate(bowl):
            if v == min_v: bowl[i] += 1

        # 공중 부양 1
        bowl = move1(bowl)

        # 물고기 수 조절
        bowl = move_fish(bowl)

        # 어항을 다시 일렬로
        bowl = stretch_bowl(bowl)

        # 공중 부양 2
        bowl = move2(bowl)

        # 물고기 수 조절
        bowl = move_fish(bowl)

        # 어항을 다시 일렬로
        bowl = stretch_bowl(bowl)

    print(ans)
